In [28]:
from glob import iglob
from datetime import datetime
from itertools import chain, repeat
from functools import lru_cache

import numpy as np
import pandas as pd
from cytoolz import compose, reduce
from h5py import File
from dask import delayed, compute
from dask.bag import from_sequence
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

In [22]:
def read_delay(filename):
    with File(filename, 'r') as f:
        key = '/user_laser/delay_line/position'
        dt = f[key][0]
        if all(dt == f[key][...]):
            return dt
        return np.nan


def read_scala(filename, tag_offset = None):
    if tag_offset is None:
        where = slice(None)
    else:
        where = slice(tag_offset, None)
    with File(filename, 'r') as f:
        try:
            for tag, iom, img, dt in zip(
                f['/bunches'][where],
                f['/photon_diagnostics/FEL01/I0_monitor/iom_sh_a_pc'][where],
                f['/vmi/andor'],
                f['/user_laser/delay_line/position'][where],
            ):
                yield {
                    'tag': tag,
                    'iom_intensity_pc': iom,
                    'img_intensity': img.sum(),
                    'delay_float': dt,
                }
        except KeyError:
            return


def read_img(filename, tag_offset = None):
    if tag_offset is None:
        where = slice(None)
    else:
        where = slice(tag_offset, None)
    with File(filename, 'r') as f:
        try:
            for tag, iom, img in zip(
                f['/bunches'][where],
                f['/photon_diagnostics/FEL01/I0_monitor/iom_sh_a_pc'][where],
                f['/vmi/andor'],
            ):
                yield {
                    'tag': tag,
                    'iom_intensity_pc': iom,
                    'img': img.astype('double'),
                }
        except KeyError:
            return

In [6]:
run = 212
tag_offset = 0  # 0 or 1
filenames = sorted(fn for fn in set(iglob(f"/data/*/Run_{run:03d}/rawdata/*.h5")))
filelist = pd.DataFrame(
    {'filename': fn, 'delay_float': read_delay(fn)}
    for fn in filenames
)
filelist

,delay_float,filename
0,0.000000,/data/Step501N2/Run_212/rawdata/Run_212_0.h5
1,NaN,/data/Step501N2/Run_212/rawdata/Run_212_486595...
2,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...
3,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...
4,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...
5,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...
6,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...
7,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...
8,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...
9,-6.999999,/data/Step501N2/Run_212/rawdata/Run_212_486596...


In [16]:
def binit(d):
    centers = np.arange(-10, 0, 0.03)  # Check delay step!
    bins = (centers[1:] + centers[:-1]) / 2
    return centers.item(np.digitize(d, bins=bins))


where = (filelist["delay"] != np.nan) & (filelist["delay"] < -1)
filelist["delay"] = filelist["delay_float"].apply(binit)
filelist = filelist[where]
filelist["delay"].unique()

array([-7.  , -6.97, -6.94, -6.91, -6.88, -6.85, -6.82, -6.79, -6.76,
       -6.73, -6.7 , -6.67, -6.64, -6.61, -6.58, -6.55, -6.52, -6.49,
       -6.46, -6.43, -6.4 , -6.37, -6.34, -6.31, -6.28, -6.25, -6.22,
       -6.19, -6.16])

In [60]:
bg_period, bg_mod = 3, 0  # /Background_Period


@lru_cache()
def sumupit(*filenames):
    ddf = (
        from_sequence(filenames)
        .map(read_img, tag_offset=tag_offset)
        .flatten()
        .to_dataframe()
    )
    ddf["is_bg"] = ddf["tag"] % bg_period == bg_mod
    where = (
        10 < ddf['iom_intensity_pc']  # Change me!
    )

    with ProgressBar():
        ret = compute([
            {
                'is_bg': bg, 'delay': dt,
                'count': delayed(ddf[where & (ddf['is_bg'] == bg)]['img'].to_bag().count()),
                'summed': delayed(ddf[where & (ddf['is_bg'] == bg)]['img'].to_bag().sum()),
            }
            for bg in [True, False]
        ])
    return list(chain(*ret))

In [62]:
grouppedlist = (
    filelist[filelist["delay"] == dt]["filename"].values
    for dt in filelist["delay"].unique()
)

summed = list(chain.from_iterable(sumupit(*fns) for fns in grouppedlist))

[########################################] | 100% Completed |  2min 14.2s
[########################################] | 100% Completed |  1min  4.1s
[########################################] | 100% Completed |  1min  4.3s
[########################################] | 100% Completed |  1min  6.1s
[########################################] | 100% Completed |  1min  3.5s
[########################################] | 100% Completed |  2min  6.9s
[########################################] | 100% Completed |  1min  2.2s
[########################################] | 100% Completed |  1min  1.8s
[########################################] | 100% Completed |  1min  6.3s
[########################################] | 100% Completed |  1min  6.7s
[########################################] | 100% Completed |  2min  7.5s
[########################################] | 100% Completed |  1min  3.4s
[########################################] | 100% Completed |  1min  1.8s
[#####################################

In [69]:
summed = pd.DataFrame(summed).summed.set_index(['delay', 'is_bg'])

count                                             summed
delay is_bg                                                          
-6.16 True    1632  [[177155.0, 176427.0, 176000.0, 178184.0, 1789...
      False   3264  [[362529.0, 362972.0, 359502.0, 364798.0, 3657...
      True    1621  [[175864.0, 175052.0, 174417.0, 177056.0, 1770...
      False   3234  [[358606.0, 358785.0, 355417.0, 360811.0, 3620...
      True    1655  [[179204.0, 178802.0, 177821.0, 180357.0, 1811...
      False   3303  [[366002.0, 366983.0, 362911.0, 368231.0, 3699...
      True    3265  [[354150.0, 353178.0, 351098.0, 355858.0, 3576...
      False   6517  [[724755.0, 724802.0, 718673.0, 728687.0, 7318...
      True    1563  [[169367.0, 168959.0, 168249.0, 170473.0, 1713...
      False   3135  [[349125.0, 350143.0, 345753.0, 350934.0, 3520...
      True    1631  [[176769.0, 176283.0, 175772.0, 177893.0, 1781...
      False   3265  [[363091.0, 363611.0, 359595.0, 365153.0, 3665...
      True    1634  [[176943.0, 177077.0, 175600.0, 178055.0, 1788...
      False   3263  [[363108.0, 363129.0, 359657.0, 364787.0, 3664...
      True    1584  [[171842.0, 171341.0, 170531.0, 172875.0, 1735...
      False   3172  [[354081.0, 354874.0, 350411.0, 355645.0, 3571...
      True    3251  [[352491.0, 351751.0, 350186.0, 353997.0, 3562...
      False   6520  [[727113.0, 728838.0, 720982.0, 731202.0, 7346...
      True    1600  [[173815.0, 173267.0, 172126.0, 174412.0, 1754...
      False   3193  [[357185.0, 357815.0, 354019.0, 358318.0, 3605...
      True    1577  [[171718.0, 170684.0, 169794.0, 171645.0, 1729...
      False   3166  [[354799.0, 354860.0, 350615.0, 355994.0, 3580...
      True    1598  [[173837.0, 173435.0, 172239.0, 174637.0, 1753...
      False   3200  [[358971.0, 358689.0, 355271.0, 359903.0, 3623...
      True    1667  [[181137.0, 180016.0, 179468.0, 181877.0, 1828...
      False   3330  [[373441.0, 372822.0, 369433.0, 374412.0, 3766...
      True    3265  [[354559.0, 353283.0, 351814.0, 356260.0, 3581...
      False   6529  [[732156.0, 733569.0, 725862.0, 734870.0, 7383...
      True    1597  [[173323.0, 172577.0, 171990.0, 174551.0, 1753...
      False   3197  [[358655.0, 359474.0, 355454.0, 360340.0, 3616...
      True    1666  [[180561.0, 179972.0, 179457.0, 182143.0, 1825...
      False   3333  [[373320.0, 373700.0, 369897.0, 374993.0, 3765...
      True    1662  [[180225.0, 179922.0, 179181.0, 181473.0, 1819...
      False   3329  [[371786.0, 373364.0, 368413.0, 373885.0, 3753...
      True    1634  [[177123.0, 176846.0, 175701.0, 178298.0, 1787...
      False   3263  [[363808.0, 363894.0, 360840.0, 365443.0, 3675...
      True    3131  [[328890.0, 327876.0, 326437.0, 330707.0, 3321...
      False   6264  [[674782.0, 673597.0, 668823.0, 678109.0, 6803...
      True    1598  [[173340.0, 173086.0, 172348.0, 174340.0, 1752...
      False   3191  [[353428.0, 353469.0, 350416.0, 355630.0, 3574...
      True    1585  [[171869.0, 171102.0, 170447.0, 172990.0, 1736...
      False   3174  [[350520.0, 350003.0, 347048.0, 352168.0, 3540...
      True    1539  [[167174.0, 166743.0, 165519.0, 167916.0, 1688...
      False   3059  [[337302.0, 337057.0, 333996.0, 338886.0, 3405...
      True    1575  [[170814.0, 171033.0, 169520.0, 172368.0, 1728...
      False   3157  [[345856.0, 345927.0, 343020.0, 347963.0, 3501...
      True    3215  [[349391.0, 348252.0, 346302.0, 351293.0, 3532...
      False   6426  [[704853.0, 703675.0, 698956.0, 708821.0, 7126...
      True    1622  [[175903.0, 175264.0, 174677.0, 177072.0, 1781...
      False   3247  [[355195.0, 354264.0, 352370.0, 357325.0, 3592...
      True    1593  [[173204.0, 173289.0, 171633.0, 174298.0, 1748...
      False   3192  [[348813.0, 348710.0, 345916.0, 351286.0, 3528...
      True    1618  [[175600.0, 175368.0, 174683.0, 176680.0, 1777...
      False   3245  [[354545.0, 353707.0, 351655.0, 356024.0, 3579...
      True     486  [[52520.0, 52412.0, 52242.0, 52885.0, 53149.0,...
      False    963  [[1

In [71]:
avg = {
    k: {'delay': k,
        'bg_n': summed.loc[(k, True), 'count'],
        'bg_img': summed.loc[(k, True), 'summed'] / summed.loc[(k, True), 'count'],
        'sg_n': summed.loc[(k, False), 'count'],
        'sg_img': summed.loc[(k, False), 'summed'] / summed.loc[(k, False), 'count'],
        'df_n': summed.loc[(k, False), 'count'],
        'df_img': (summed.loc[(k, False), 'summed'] / summed.loc[(k, False), 'count']
                   - summed.loc[(k, True), 'summed'] / summed.loc[(k, True), 'count'])}
    for k in filelist["delay"].unique()
} 

for k, d in avg.items():
    plt.figure(figsize=(15, 5))
    plt.subplot(131)
    plt.title("Background")
    plt.pcolormesh(d['bg_img'], cmap="Greys")
    plt.clim(0, None)
    plt.axis('equal')
    plt.subplot(132)
    plt.title("Signal")
    plt.pcolormesh(d['sg_img'], cmap="Greys")
    plt.clim(0, None)
    plt.axis('equal')
    plt.subplot(133)
    plt.title("Diff")
    plt.pcolormesh(d['df_img'], cmap="Greys")
    plt.clim(0, None)
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    break

plt.figure()
plt.plot([k for k in avg], [d['df_img'].sum() for d in avg.values()], 'o-')
plt.grid(True)
plt.show()

KeyError: (-7.000000000000064, True)

In [21]:
# print(filenames)
writein = f"/data/Step501N2/Run_{run:03d}/work"

for k, d in avg.items():
    with File(f"{writein}/reduced_dt={k:.3f}.h5", "w") as f:
        f['delay'] = d['delay']
        f['bg_n'] = d['bg_n']
        f['bg_img'] = d['bg_img']
        f['sg_n'] = d['sg_n']
        f['sg_img'] = d['sg_img']
        f['df_n'] = d['df_n']
        f['df_img'] = d['df_img']